# Demo 1: Brno 2023

Reálná situace v Brně na jaře 2023: 137 školek a přibližně 3200 dětí. 

## Výroba datasetu
Skript vygeneruje 3200 hráčů-uchazečů ve věku 2 až 7 let s náhodným datem narození, spádovou školkou a dalšími atributy převzatými z reálného přihlašovacího formuláře z webu zapisdoms.brno.cz. 


In [14]:

import csv
import datetime
from dateutil.relativedelta import relativedelta
import random
import pandas as pd
import copy

random.seed(11)

skolky_spadove = 135    # běžné školky
skolky_vse = 137   # 2 školky mají jako spádovou oblast celé Brno

def get_random_birthday():    # vymyslet korekci - zvysit pocet 3 a 4 letych na ukor starsich - ti uz v nejake skolce jsou z vetsiny...
    # Fuknce generuje náhodná data narození dětí ve věku 2 až 7 let. 
    today = datetime.date.today()
    current_year = today.year
    birthday_range = datetime.date(current_year-2, 8, 30) - datetime.date(current_year-7, 5, 31)    # (current_year-7, 5, 31) je vysledkem odhadu
    random_integer = random.randint(0, birthday_range.days)
    random_birthday = datetime.date(current_year-2, 8, 30) - relativedelta(days = random_integer)
    return random_birthday
random_birthday = get_random_birthday()

def get_random_school():
    # Funkce generuje náhodnou spádovou školku. U desetiny uchazečů vygeneruje také školku, kam chodí starší sourozenec.
    spadova_skolka = random.randint(0, skolky_spadove-1)   
    random_number = random.randint(1, 1000)
    if random_number%10 == 0:                         
        skolka_sourozence = spadova_skolka
    if random_number%20 == 0:
        skolka_sourozence = random.randint(1, skolky_vse-1)  
    else:
        skolka_sourozence = None                 
    return spadova_skolka, skolka_sourozence

je_bydliste_Brno = lambda x: False if x%21 == 0 else True  
# Funkce u většiny uchazečů určí jako místo bydliště Brno. Počet brněnských vs. mimobrněnských je výsledkem odhadu.

je_prodlouzena_dochazka = lambda x: True if x%33 == 0 else False   
# Funkce u většiny dětí zvolí, že nepožadují prodlouženou docházku. Počet je opět vysledkem odhadu. 

def get_names(jmena_divky, jmena_hosi):
    # vyrobi jména fiktivním uchazečům. Vstupem jsou nejběžnější česká a slovenská křestní jména a příjmení.
    first_names_0, surnames_0 = [], []
    with open(jmena_divky, newline='') as file:
        fi = csv.reader(file, delimiter=';')
        for line in fi:
            first_names_0.append(line[0])
            surnames_0.append(line[1])
    girls = [{'jmeno': i + ' ' + ii} for i in first_names_0 for ii in surnames_0]

    first_names_1, surnames_1 = [], []
    with open(jmena_hosi, newline='') as file:
        f = csv.reader(file, delimiter=';')
        for line in f:
            first_names_1.append(line[0])
            surnames_1.append(line[1])
    boys = [{'jmeno': i + ' ' + ii} for i in first_names_1 for ii in surnames_1]

    return girls, boys
    
girls, boys = get_names('dummy_input_data/divky.csv', 'dummy_input_data/hosi.csv')

for i in boys:
    i['pohlavi'] = 'M'
for i in girls:
    i['pohlavi'] = 'F'

children = boys + girls

for i in children:
    i['datum_narozeni'] = get_random_birthday()
    i['spadova_skolka'] = get_random_school()[0]
    i['skolka_sourozence'] = get_random_school()[1]
    i['bydliste_brno'] = je_bydliste_Brno(random.randint(1, 1000))
    i['prodlouzena_dochazka'] = je_prodlouzena_dochazka(random.randint(1, 1000))

print("Náhodně vybraný uchazeč: ")
random.seed(None)
print(random.choice(children))


Náhodně vybraný hráč: 
{'jmeno': 'Tereza Tesařová', 'pohlavi': 'F', 'datum_narozeni': datetime.date(2017, 1, 15), 'spadova_skolka': 28, 'skolka_sourozence': None, 'bydliste_brno': True, 'prodlouzena_dochazka': False}


## Uchazeči si vybírají školky
Každý uchazeč se může přihlásit, do kolika školek chce. Volí si je formou prioritního seznamu, kde nejoblíbenější školka je na pozici 1 a tak dál. 

In [23]:

random.seed(12)

def get_priorities(deti_list, pocet_skolek):
    # Fuknce simuluje výběr oblíbených školek tím, že přiřadí uchazeči náhodný seznam. Počet položek je také náhodný.
    # Funkce zohledňuje spádovost a školku sourozence.
    with open('prihlasky.csv', 'w') as file:
        writer = csv.DictWriter(file, fieldnames = ['dite', 'skolka', 'poradi'])
        writer.writeheader()
        for _ in range(len(deti_list)):
            vybrane_skolky = set()   
            pocet_prihlasek = random.randint(1, pocet_skolek)    # tady lze omezit, kolik přihlášet může každý podat
            for i in range(pocet_prihlasek):
                if i == 0:
                    vybrane_skolky.add(str(deti_list[_]['spadova_skolka']))
                if (i == 1 and deti_list[_]['skolka_sourozence'] != None):
                    vybrane_skolky.add(str(deti_list[_]['skolka_sourozence']))
                else:
                    vybrane_skolky.add(str(random.randint(0, pocet_skolek-1)))
            vybrane_skolky = list(vybrane_skolky) 
            for index, value in enumerate(vybrane_skolky):
                d = {'dite': str(_), 'skolka': value, 'poradi': index + 1}
                writer.writerow(d)
        return
get_priorities(children, skolky_vse)

children_df = pd.DataFrame(children)
new_col = children_df.index
children_df.insert(0, 'id_dite', new_col)
# children_df.to_csv('deti.csv')


Ve výsledku má každý uchazeč k sobě přiřazený seznam svých školek, seřazený podle oblíbenosti. Uchazeči i školky jsou reprezentovány svým ID:

In [27]:
def get_priorities():
    prihlasky_df = pd.read_csv('prihlasky.csv')    # toto neni nutne... ukladat do a pak zase otevirat z CSV
    prihlasky_copy = prihlasky_df.copy()
    prihlasky_copy['dite'] = prihlasky_copy['dite'].astype('str')
    prihlasky_copy['skolka'] = prihlasky_copy['skolka'].astype('str')
    prihlasky_groupedby_kids = prihlasky_copy.groupby('dite')['skolka'].agg(list)
    return prihlasky_groupedby_kids       # ==================== ko. 34?  Vyrobit rovnou demo 2 s miniseznamem 
priorities = get_priorities()
print(priorities)


dite
0       [34, 4, 95, 109, 102, 36, 128, 63, 88, 131, 11...
1       [33, 100, 34, 50, 4, 14, 36, 128, 63, 88, 90, ...
10      [34, 100, 50, 95, 81, 109, 14, 102, 128, 63, 8...
100     [33, 100, 4, 81, 14, 102, 128, 90, 118, 67, 11...
1000    [34, 4, 102, 36, 128, 63, 88, 115, 90, 118, 6,...
                              ...                        
995     [33, 48, 35, 4, 20, 58, 12, 65, 72, 54, 112, 6...
996     [117, 42, 119, 0, 73, 124, 89, 120, 65, 46, 14...
997     [34, 100, 101, 19, 93, 124, 81, 44, 121, 25, 6...
998     [34, 66, 35, 109, 102, 63, 59, 75, 73, 130, 12...
999     [74, 100, 48, 70, 66, 35, 50, 4, 135, 109, 107...
Name: skolka, Length: 3200, dtype: object
